## Загрузка датасета

In [ ]:
!mkdir datasets
!mkdir models

Ссылка на скачивание датасета Spider на GitHub https://github.com/lumia-group/rasat

На https://github.com/taoyds/spider содержится его препроцессинг

In [1]:
!mkdir spider_data

In [2]:
!git clone https://github.com/taoyds/spider spider_data

Cloning into 'spider_data'...
remote: Enumerating objects: 383, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 383 (delta 17), reused 13 (delta 13), pack-reused 356
Receiving objects: 100% (383/383), 44.93 MiB | 2.81 MiB/s, done.
Resolving deltas: 100% (111/111), done.
Updating files: 100% (261/261), done.


In [3]:
import json
import nltk
from spider_data.process_sql import get_schema, Schema, get_sql

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/mat/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
with open('datasets/spider/train_spider.json') as file:
    spider_train = json.load(file)

In [5]:
spider_train[0]

{'db_id': 'department_management',
 'query': 'SELECT count(*) FROM head WHERE age  >  56',
 'query_toks': ['SELECT',
  'count',
  '(',
  '*',
  ')',
  'FROM',
  'head',
  'WHERE',
  'age',
  '>',
  '56'],
 'query_toks_no_value': ['select',
  'count',
  '(',
  '*',
  ')',
  'from',
  'head',
  'where',
  'age',
  '>',
  'value'],
 'question': 'How many heads of the departments are older than 56 ?',
 'question_toks': ['How',
  'many',
  'heads',
  'of',
  'the',
  'departments',
  'are',
  'older',
  'than',
  '56',
  '?'],
 'sql': {'from': {'table_units': [['table_unit', 1]], 'conds': []},
  'select': [False, [[3, [0, [0, 0, False], None]]]],
  'where': [[False, 3, [0, [0, 10, False], None], 56.0, None]],
  'groupBy': [],
  'having': [],
  'orderBy': [],
  'limit': None,
  'intersect': None,
  'union': None,
  'except': None}}

In [6]:
db_path = "datasets/spider/database/"

def get_schema_by_name(name):
    return get_schema(db_path + name + '/' + name + ".sqlite")

def get_schema_str(name):
    schema_json = get_schema_by_name(name)

    schema_str = ""

    for header in schema_json:
        schema_str += f"header {header} : "
        schema_str += " | ".join(schema_json[header])
        schema_str += '\n'

    schema_str = schema_str[:-1]

    return schema_str

def make_schema(headers):
    return Schema({"table" : headers})

In [10]:
spider_data_ready = []

for sample in spider_train:
    question = sample["question"]

    headers = get_schema_str(sample["db_id"])

    sentence = "translate to SQL: " + question + "\n\n" + headers

    query = sample["query"]

    spider_data_ready.append({"question" : sentence, "query" : query})

In [11]:
with open("datasets/spider_dataset.json", "w") as f:
    json.dump(spider_data_ready, f)

In [15]:
spider_data_ready[0]

{'question': 'translate to SQL: How many heads of the departments are older than 56 ?\n\nheader department : department_id | name | creation | ranking | budget_in_billions | num_employees\nheader head : head_id | name | born_state | age\nheader management : department_id | head_id | temporary_acting',
 'query': 'SELECT count(*) FROM head WHERE age  >  56'}